In [1]:
import pandas as pd
import re
from sklearn.metrics import f1_score, cohen_kappa_score
from tqdm import tqdm

In [2]:
#For arabic
files = ['PA_base']

def map_fn(row):
    row = str(row)
    if(re.search('نعم', row) and re.search('لا', row)):
        return None
    elif(re.search('نعم', row)):
        return 1
    elif(re.search('لا', row)):
        return 0

    return None

In [4]:
#For hindi
files = ['PH_base']

def map_fn(row):
    row = str(row)
    if(re.search('हाँ', row) or re.search('हां', row) or re.search('जी', row)):
        return 1
    elif(re.search('नहीं', row) or re.search('गलत', row)):
        return 0

    return None

In [5]:
for f in tqdm(files):
    df = pd.read_excel('../OS_LLMs/'+f+'.xlsx')
    col = df.columns[3:]
    
    no_ans = []
    F1 = []
    IAA = []
    HS = []
    
    for i in col:
        df[i] = df[i].apply(lambda x: str(x).lower())
        df[i] = df[i].apply(map_fn)

        pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
        no_ans.append(df[i].isna().sum())

        tmp = df[['label', i]].dropna()

        F1.append(pen*f1_score(tmp['label'], tmp[i], average='weighted'))
        IAA.append(pen*cohen_kappa_score(tmp['label'], tmp[i]))
        HS.append(tmp[tmp[i]==1].shape[0]/tmp.shape[0])
    
    res = pd.read_csv('Performance_LocalLLMs.csv')
    res[f+' Hal'] = no_ans
    res[f+' F1'] = F1
    res[f+' IAA'] = IAA
    #res[f+' HS'] = HS
    res.to_csv('Performance_LocalLLMs.csv', index=False)
    
#no_ans[-1], F1[-1], IAA[-1], HS[-1]

100%|██████████| 1/1 [00:00<00:00, 10.52it/s]


In [3]:
# Sample code check mapping from words to labels

for f in files:
    df = pd.read_excel('../OS_LLMs/'+f+'.xlsx')

    print(df.gpt.value_counts())
    print(df.gpt.value_counts().index)

gpt
نعم، ال       121
لا، البي      119
أعتذر          96
آسف،           42
أنا آ          25
البيان الم     17
أعتقد          10
لا، لا ي        8
نعم، ي          8
عذرً            7
نعم، الت        5
شكرً            5
أنا مس          3
أنا مج          3
لا يجوز         3
أنا أع          2
التحرش          2
لا يوجد         2
لا يمكن         2
صباح ال         1
يجب تج          1
لا يكره         1
رعي الغ         1
نعم، أ          1
لا أستطي        1
أنا بر          1
بالعكس          1
نعم, ال         1
بارك الل        1
المال ز         1
الدين الإ       1
البيان ال       1
لا، ليس         1
أنا لا          1
لا، شمام        1
اللاجئ          1
لا يجب است      1
في بلاد         1
أنا أف          1
Name: count, dtype: int64
Index(['نعم، ال', 'لا، البي', 'أعتذر', 'آسف،', 'أنا آ', 'البيان الم', 'أعتقد',
       'لا، لا ي', 'نعم، ي', 'عذرً', 'نعم، الت', 'شكرً', 'أنا مس', 'أنا مج',
       'لا يجوز', 'أنا أع', 'التحرش', 'لا يوجد', 'لا يمكن', 'صباح ال',
       'يجب تج', 'لا يكره', 'رعي الغ', 'ن